In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

import tensorflow as tf

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import matthews_corrcoef, confusion_matrix
from sklearn.utils import class_weight
from skimage import exposure

from scipy.stats import skew, kurtosis

import numpy as np
from datetime import datetime, date, time, timedelta
from dateutil import parser
import csv

import itertools

import h5py
import os, fnmatch

In [ ]:
'''
fracdim part based on https://stackoverflow.com/questions/44793221/python-fractal-box-count-fractal-dimension
Not returning skewdness or kurtosis here but user can add these in 
and adjust corresponding parts of code further on in the rest of the notebook ###
'''

def stats_mbfracdim(data, threshold, switch):
    
    # 7 stats part
    min_data = np.nanmin(data)
    max_data = np.nanmax(data)
    mean_data = np.nanmean(data)
    median_data = np.nanmedian(data)
    std_data = np.nanstd(data)
    if len(np.where(np.array(data).flatten() != np.array(data).flatten())[0]) > 0:
        skew_data = float(skew(data.flatten(),nan_policy='omit').data)
    else:
        skew_data = skew(data.flatten(),nan_policy='omit')
    kurtosis_data = kurtosis(data.flatten(),nan_policy='omit')
    
    # fracdim part: Only for 2d image
    assert(len(data.shape) == 2)

    ind_nan = np.where(np.array(data.flatten()) != np.array(data.flatten()))[0]
    data.flat[ind_nan] = 0.

    def boxcount(data, k):
        S = np.add.reduceat(
                np.add.reduceat(data, np.arange(0, data.shape[0], k), axis=0),
                       np.arange(0, data.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])

    # Transform data into a binary array
    if switch == 1:
        data = (data < threshold)
    elif switch == -1:
        data = (data > threshold)

    # Minimal dimension of image
    p = np.min(data.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(data, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    
    return np.array([max_data, min_data, mean_data, std_data, -coeffs[0], 0])
    ### 0 is included as the last entry of the returned array in order to be able to exclude features too

In [ ]:
"""
FIND SPECIFIED PATTERN USING FNMATCH 
"""
def pattern_finder(home_dir, pattern):
    
    names = []
    for root, dirs, files in os.walk(home_dir):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                name = str(name)
                names.append(name)
    
    return names

The objective of the deep learning system investigated in this project is the forecast of the N-S component of the IMF, Bz, measured in the GSM coordinate system at Lagrange point 1 (L1), in a two day window, 3 - 5 days ahead of the corresponding Solar and Heliophysics Observatory (SOHO) image products.
This challenge is cast as a binary classification problem and compared with a Gaussian Naive Bayes classifier baseline. 
The value computed is not the raw Bz from OMNI but rather we define the new Bz as the min of raw Bz minus the mean of raw Bz in the above mentioned two-day window.
The ground truth labels are obtained from the low-resolution, hourly-averaged values are obtained from the **OMNI** database: [OMNI2](https://omniweb.gsfc.nasa.gov/html/ow_data.html). The time range from this OMNI set that has been made available here is from Jan. 1st 1999 till Dec. 31st 2010. 

Provide the path to the downloaded OMNI data set to the ```np.genfromtxt()``` function in the cell below. 
The following definition for the 'nulls' variable comes from Section 3 of '**_Description of records and words_**' from the 'Fill values' column. There are a total of 57 columns listed but only 55 columns are actually present since 'F9.6' and 'F7.4' (columns 55 and 56, respectively) are omitted in the data set. Descriptions of the data are provided at the above NASA url. This step is performed to homogenize the representations of all values that are missing in the data set. Although this step is not strictly needed for our use-case here, it is useful if the user wished to use a different variable as their ground truth label. 

In [ ]:
PATH_TO_OMNI_DATA = '/home/carl/Documents/OMNI_DATA/'
omni2_data_raw = np.genfromtxt(f'{PATH_TO_OMNI_DATA}omni2_1999-01-01_to_2010-12-31.txt')
nulls = np.array([np.nan, np.nan, np.nan, 9999, 99, 99, 999, 999, 999.9, 999.9, 999.9, 999.9,
    999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9, 999.9,
    9999999., 999.9, 9999., 999.9, 999.9, 9.999, 99.99, 9999999., 999.9, 9999.,
    999.9, 999.9, 9.999, 999.99, 999.99, 999.9, 99, 999, 99999, 9999,
    999999.99, 99999.99, 99999.99, 99999.99, 99999.99, 99999.99, 0, 999, 999.9,
    999.9, 99999, 99999, 99.9
])
print('len(nulls):', len(nulls))
omni2_data_nan_processed = np.where(omni2_data_raw == nulls[None, :], np.nan, omni2_data_raw)
print('shape(omni2_data_nan_processed):', np.shape(omni2_data_nan_processed))

Convert OMNI date-time format of 'Year-Day_of_Year-Time_of_Day' to date-time object.

Choose the Bz GSM column which is the 16th column (counting the pythonic way from 0)

Identify dates for which the ground truth label is missing.

**Note**: If there would be missing ground truth labels that would influence our approach, we would need to account for this by removing corresponding SOHO images from the data cubes. From the procedure used in this project to arrive at the computed Bz value, these missing raw Bz values do not require any modification to the pipeline produced image cubes in this case. In another ground truth label setting or forecasting window, they might.  

In [ ]:
omni2_date = [datetime(int(yr),1,1) + timedelta(days = int(day) - 1) + timedelta(hours = hr) for yr,day,hr in np.vstack((omni2_data_nan_processed.T[0],omni2_data_nan_processed.T[1], omni2_data_nan_processed.T[2])).T ]

omni2_Bz = omni2_data_nan_processed.T[16]

missing_Bz_vals_ind = np.where(np.array(omni2_Bz) != np.array(omni2_Bz))[0]
print('Missing ground truth labels from Jan. 1st 1999 to Dec. 31st 2010 in data:\n', [str(omni2_date[ind]) for ind in missing_Bz_vals_ind])
print('Number of missing ground truth labels:', len(missing_Bz_vals_ind))

Convert above transformed OMNI date-time object to a string for efficient matching when comparing with synced string date-times.

In [ ]:
omni2_date_copy_str = [str(elem) for elem in omni2_date]

Provide the path to the synced times contained in the **CSV** file for which to obtain the ground truth labels and specify ```dtype = 'str'```. 
As a check, ensure that the output length corresponds to the length of the CSV file which you expect to have.  

In [ ]:
'''
Need to adjust input according to if have 1, 3, or 7 SOHO products
'''

path_to_synced_times = '/home/carl/Documents/synced_csvs_1_3_7fams/'
synced_times_file_name = '1999-01-01_to_2010-12-31_EIT304_6_6_128_times_sync_All7.csv'
#'1999-01-01_to_2010-12-31_MDI_96m_6_6_128_times_sync_onlyMDI.csv'
#'1999-01-01_to_2010-12-31_MDI_96m_6_6_128_times_sync_3products.csv'
synced_times = np.genfromtxt(f'{path_to_synced_times}{synced_times_file_name}',dtype = 'str')
print('len(synced_times):', len(synced_times))

Convert '19990202160302' string date-time format to a date-time object and compute the value of the date-time object 3 days ahead of the time found after rounding to the nearest hour and convert to string. Rounding is done to enable direct comparison with the OMNI date-time objects.

In [ ]:
synced_times_datetimes = [parser.parse(elem) for elem in synced_times]
synced_times_datetimes_rounded = [elem.replace(second = 0, microsecond = 0, minute = 0, hour = elem.hour) + timedelta(hours = elem.minute//30) for elem in synced_times_datetimes]
synced_times_datetimes_rounded_3days_ahead = np.array(synced_times_datetimes_rounded) + timedelta(days = 3)
synced_times_datetimes_rounded_3days_ahead_copy_str = [str(elem) for elem in synced_times_datetimes_rounded_3days_ahead]

Now can compare synced times directly with the times from OMNI data and compute Bmin - Bmean, 3-5 days ahead of the SOHO image product times. The figure of 48 is added in order to arrive at the 5 day mark of the two-day window. Need to use np.nanmin and np.nanmean as a result of the 24 missing raw Bz values. This is the most time consuming step thus far. 

In [ ]:
omni2_3days_ahead_ind = [np.where(np.array(omni2_date_copy_str) == elem)[0][0] for elem in synced_times_datetimes_rounded_3days_ahead_copy_str]
omni2_Bzmin_minus_mean_3to5days_ahead = [ np.round(np.nanmin(omni2_Bz[elem: elem + 48]) - np.nanmean(omni2_Bz[elem: elem + 48]),2) for elem in omni2_3days_ahead_ind]

Option to output the computed Bz ground truth labels corresponding to the synced times to a new CSV file in the same directory as the synced times. The user can change the name for the CSV file that is to be output.

In [ ]:
'''
User should provide a more descriptive name to include the types of synced products for the name of the CSV file
or save the CSV file in a different directory

Optional
'''

with open(f'{path_to_synced_times}Bz_min_minus_mean_3to5days_ahead.csv', 'a') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(omni2_Bzmin_minus_mean_3to5days_ahead)

In [ ]:
'''
Freeing the memory taken up by this variable
'''
omni2_data_nan_processed = None 

Constructing the 'rolling data split'. In first year of data: training is Feb. - Aug., validation is Oct. and testing is Nov. Each subsequent year these periods are cyclically moved forward by one month.

In [ ]:
'''
Building up the rolling indices scheme
'''

synced_times_datetimes_rounded_years = np.array([elem.year for elem in synced_times_datetimes_rounded])
synced_times_datetimes_rounded_months = np.array([elem.month for elem in synced_times_datetimes_rounded])

start_ind_train = []
end_ind_train = []

start_ind_valid = []
end_ind_valid = []

start_ind_test = []
end_ind_test = []


# Hardcoded: Feb [0] - Aug [6], Oct [8], Nov [9] in Python index for start of year in new_month_order sequence
# In our case will have 18 indices because of splitting of indices to obtain proper start and end indices

month_order = np.arange(1,12+1)

counter = 1
for yr in range(synced_times_datetimes_rounded_years[0],synced_times_datetimes_rounded_years[-1] + 1):
    new_month_order = np.roll(month_order,-counter) #since we start from Feb.1999 so roll back year by 1 (i.e., -1) 
    
    if new_month_order[6] < new_month_order[0]: # Aug. is [6] in the first new_month_order for 1999.
        pos_one = np.where(new_month_order == 1)[0] # 1 is the location of Jan.
        ind_start_train_part1 = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[pos_one]))[0][0]
        ind_end_train_part1 = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[pos_one + (6-pos_one)]))[0][-1] #6 is for Aug. position
        
        ind_start_train_part2 = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[0]))[0][0]
        ind_end_train_part2 = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[pos_one-1]))[0][-1]

        start_ind_train.append(ind_start_train_part1) 
        start_ind_train.append(ind_start_train_part2)
        end_ind_train.append(ind_end_train_part1)
        end_ind_train.append(ind_end_train_part2)
        
        
        ind_start_valid = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[8]))[0][0] 
        start_ind_valid.append(ind_start_valid)

        ind_end_valid = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[8]))[0][-1]
        end_ind_valid.append(ind_end_valid)


        ind_start_test = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[9]))[0][0]
        start_ind_test.append(ind_start_test)

        ind_end_test = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[9]))[0][-1]
        end_ind_test.append(ind_end_test) 

        
    else:
    
        ind_start_train = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[0]))[0][0]
        ind_end_train = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[6]))[0][-1]
        
        start_ind_train.append(ind_start_train)
        end_ind_train.append(ind_end_train)
        
    
        ind_start_valid = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[8]))[0][0] 
        start_ind_valid.append(ind_start_valid)

        ind_end_valid = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[8]))[0][-1]
        end_ind_valid.append(ind_end_valid)


        ind_start_test = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[9]))[0][0]
        start_ind_test.append(ind_start_test)

        ind_end_test = np.where((synced_times_datetimes_rounded_years == yr) & (synced_times_datetimes_rounded_months == new_month_order[9]))[0][-1]
        end_ind_test.append(ind_end_test)       
        
    print('yr:', yr)
    counter += 1


diff_train = np.array(end_ind_train) - np.array(start_ind_train)

diff_valid = np.array(end_ind_valid) -  np.array(start_ind_valid)

diff_test = np.array(end_ind_test) - np.array(start_ind_test)

diff_train_sum = np.sum(diff_train)
print('training data size:', diff_train_sum)

diff_valid_sum = np.sum(diff_valid)
print('validation data size:', diff_valid_sum)

diff_test_sum = np.sum(diff_test)
print('test data size:', diff_test_sum)

Data label statistics

In [ ]:
print('total data avilable:', len(synced_times))
print('expected amount of available data used in 7 months:', int(np.round((7./12)*len(synced_times_file_name),0))) #7 months of the year for training
data_total = diff_train_sum + diff_valid_sum + diff_test_sum
print('actual total data used:', data_total)
print('perc. data used from total data available:', data_total/len(synced_times_file_name))
print('perc. train:', diff_train_sum/data_total)
print('perc. valid:', diff_valid_sum/data_total)
print('perc. test:', diff_test_sum/data_total)

Reading in the HDF5 data cubes

In [ ]:
'''
Manually supply path to folder containing the synced HDF5 data cubes  

Need to adjust input according to if haave 1, 3, or 7 SOHO products
'''

path_to_synced_HDF5 = '/home/carl/Documents/juniper_datacubes_3fams/All7SOHOProducts_19990101_20101231_6_6_128/'
#'/home/carl/Documents/juniper_datacubes_3fams/MDI_19990101_20101231_6_6_128/'
#'/home/carl/Documents/juniper_datacubes_3fams/All7SOHOProducts_19990101_20101231_6_6_128/'
#'/home/carl/Documents/juniper_datacubes_3fams/MDI_EIT195_C2_19990101_20101231_6_6_128/'
name_list = pattern_finder(path_to_synced_HDF5, pattern = '*sync.h5')
print('cube name list:', name_list)
print('len(name_list):', len(name_list))

In [ ]:
cube_list = []
product_name_list = []
for cu in name_list:
    data_set = h5py.File(f'{path_to_synced_HDF5}{cu}','r')
    data_set_keys = list(data_set.keys())[0]
    if 'EIT' not in data_set_keys:
        product_name_from_keys = ''.join(data_set_keys[:-1].split('_')[0:2]) 
    else:
        product_name_from_keys = ''.join(data_set_keys[:-1].split('_')[0])        
    print(data_set_keys)
    data = data_set[str(list(data_set.keys())[0])][:]
    print(np.shape(data))
    print(np.where(data != data)[0])
    
    # Equalization
    img_eq = exposure.equalize_adapthist(data[10], clip_limit=0.03)
    
    # Showing a representative image from each data cube: slice #10
    plt.figure()
    plt.imshow(img_eq, cmap='jet')
    print('np.shape(data):', np.shape(data))
    
    #img_size = np.shape(data)[2]
    #print('img_size:', img_size)
    
    cube_list.append(data)
    product_name_list.append(product_name_from_keys)

print('product_name_list', product_name_list)
print('np.shape(cube_list):', np.shape(cube_list))

First, combining the data from the rolling indices calculated in the preceding step

In [ ]:
'''
Need to adjust input according to if haave 1, 3, or 7 SOHO products by uncommenting the corresponding
lines in each of the two blocks in each of the three loops over the train, validate, and test sets

For each SOHO product, the test set is normalized by the maximum product value found from training set 

Dimensions expanded to for batch dimension of tf tensor
'''

train_indices = list(zip(start_ind_train, end_ind_train))
valid_indices = list(zip(start_ind_valid, end_ind_valid))
test_indices = list(zip(start_ind_test, end_ind_test))

#print('len(train_indices):', len(train_indices))
#print('len(valid_indices):', len(valid_indices))
#print('len(test_indices):', len(test_indices))

SOHO_grand_train_set_1 = []
SOHO_grand_train_set_2 = []
SOHO_grand_train_set_3 = []
SOHO_grand_train_set_4 = []
SOHO_grand_train_set_5 = []
SOHO_grand_train_set_6 = []
SOHO_grand_train_set_7 = []

SOHO_grand_valid_set_1 = []
SOHO_grand_valid_set_2 = []
SOHO_grand_valid_set_3 = []
SOHO_grand_valid_set_4 = []
SOHO_grand_valid_set_5 = []
SOHO_grand_valid_set_6 = []
SOHO_grand_valid_set_7 = []

SOHO_grand_test_set_1 = []
SOHO_grand_test_set_2 = []
SOHO_grand_test_set_3 = []
SOHO_grand_test_set_4 = []
SOHO_grand_test_set_5 = []
SOHO_grand_test_set_6 = []
SOHO_grand_test_set_7 = []


Bz_train_set = []
Bz_valid_set = []
Bz_test_set = []


for i,j in train_indices:

    
    SOHO_grand_train_set_1 += list(cube_list[0][i:j])
    SOHO_grand_train_set_2 += list(cube_list[1][i:j])
    SOHO_grand_train_set_3 += list(cube_list[2][i:j])
    SOHO_grand_train_set_4 += list(cube_list[3][i:j])
    SOHO_grand_train_set_5 += list(cube_list[4][i:j])
    SOHO_grand_train_set_6 += list(cube_list[5][i:j])
    SOHO_grand_train_set_7 += list(cube_list[6][i:j])
    Bz_train_set += list(omni2_Bzmin_minus_mean_3to5days_ahead[i:j])

SOHO_grand_train_set_1_max = max(np.array(SOHO_grand_train_set_1).flat)
SOHO_grand_train_set_2_max = max(np.array(SOHO_grand_train_set_2).flat)
SOHO_grand_train_set_3_max = max(np.array(SOHO_grand_train_set_3).flat)
SOHO_grand_train_set_4_max = max(np.array(SOHO_grand_train_set_4).flat)
SOHO_grand_train_set_5_max = max(np.array(SOHO_grand_train_set_5).flat)
SOHO_grand_train_set_6_max = max(np.array(SOHO_grand_train_set_6).flat)
SOHO_grand_train_set_7_max = max(np.array(SOHO_grand_train_set_7).flat)
                                 
SOHO_grand_train_set_1_normed = np.expand_dims(SOHO_grand_train_set_1/SOHO_grand_train_set_1_max, axis=1)
SOHO_grand_train_set_2_normed = np.expand_dims(SOHO_grand_train_set_2/SOHO_grand_train_set_2_max, axis=1)
SOHO_grand_train_set_3_normed = np.expand_dims(SOHO_grand_train_set_3/SOHO_grand_train_set_3_max, axis=1)
SOHO_grand_train_set_4_normed = np.expand_dims(SOHO_grand_train_set_4/SOHO_grand_train_set_4_max, axis=1)
SOHO_grand_train_set_5_normed = np.expand_dims(SOHO_grand_train_set_5/SOHO_grand_train_set_5_max, axis=1)
SOHO_grand_train_set_6_normed = np.expand_dims(SOHO_grand_train_set_6/SOHO_grand_train_set_6_max, axis=1)
SOHO_grand_train_set_7_normed = np.expand_dims(SOHO_grand_train_set_7/SOHO_grand_train_set_7_max, axis=1)


for i,j in valid_indices:
    
    SOHO_grand_valid_set_1 += list(cube_list[0][i:j])
    SOHO_grand_valid_set_2 += list(cube_list[1][i:j])
    SOHO_grand_valid_set_3 += list(cube_list[2][i:j])
    SOHO_grand_valid_set_4 += list(cube_list[3][i:j])
    SOHO_grand_valid_set_5 += list(cube_list[4][i:j])
    SOHO_grand_valid_set_6 += list(cube_list[5][i:j])
    SOHO_grand_valid_set_7 += list(cube_list[6][i:j])   
    Bz_valid_set += list(omni2_Bzmin_minus_mean_3to5days_ahead[i:j])

SOHO_grand_valid_set_1_normed = np.expand_dims(SOHO_grand_valid_set_1/SOHO_grand_train_set_1_max, axis=1)
SOHO_grand_valid_set_2_normed = np.expand_dims(SOHO_grand_valid_set_2/SOHO_grand_train_set_2_max, axis=1)
SOHO_grand_valid_set_3_normed = np.expand_dims(SOHO_grand_valid_set_3/SOHO_grand_train_set_3_max, axis=1)
SOHO_grand_valid_set_4_normed = np.expand_dims(SOHO_grand_valid_set_4/SOHO_grand_train_set_4_max, axis=1)
SOHO_grand_valid_set_5_normed = np.expand_dims(SOHO_grand_valid_set_5/SOHO_grand_train_set_5_max, axis=1)
SOHO_grand_valid_set_6_normed = np.expand_dims(SOHO_grand_valid_set_6/SOHO_grand_train_set_6_max, axis=1)
SOHO_grand_valid_set_7_normed = np.expand_dims(SOHO_grand_valid_set_7/SOHO_grand_train_set_7_max, axis=1)

for i,j in test_indices:
    
    SOHO_grand_test_set_1 += list(cube_list[0][i:j])
    SOHO_grand_test_set_2 += list(cube_list[1][i:j])
    SOHO_grand_test_set_3 += list(cube_list[2][i:j])
    SOHO_grand_test_set_4 += list(cube_list[3][i:j])
    SOHO_grand_test_set_5 += list(cube_list[4][i:j])
    SOHO_grand_test_set_6 += list(cube_list[5][i:j])
    SOHO_grand_test_set_7 += list(cube_list[6][i:j])     
    Bz_test_set += list(omni2_Bzmin_minus_mean_3to5days_ahead[i:j])
    
SOHO_grand_test_set_1_normed = np.expand_dims(SOHO_grand_test_set_1/SOHO_grand_train_set_1_max, axis=1)
SOHO_grand_test_set_2_normed = np.expand_dims(SOHO_grand_test_set_2/SOHO_grand_train_set_2_max, axis=1)
SOHO_grand_test_set_3_normed = np.expand_dims(SOHO_grand_test_set_3/SOHO_grand_train_set_3_max, axis=1)
SOHO_grand_test_set_4_normed = np.expand_dims(SOHO_grand_test_set_4/SOHO_grand_train_set_4_max, axis=1)
SOHO_grand_test_set_5_normed = np.expand_dims(SOHO_grand_test_set_5/SOHO_grand_train_set_5_max, axis=1)
SOHO_grand_test_set_6_normed = np.expand_dims(SOHO_grand_test_set_6/SOHO_grand_train_set_6_max, axis=1)
SOHO_grand_test_set_7_normed = np.expand_dims(SOHO_grand_test_set_7/SOHO_grand_train_set_7_max, axis=1)

#double checking that same length train, validation, and test sets as previously found.   
print('np.shape(SOHO_grand_train_set_1):', np.shape(SOHO_grand_train_set_1))
#print(len(SOHO_grand_train_set_1) + len(SOHO_grand_train_set_2) + len(SOHO_grand_train_set_3)) #+ len(SOHO_grand_train_set_4) + len(SOHO_grand_train_set_5) + len(SOHO_grand_train_set_6) + len(SOHO_grand_train_set_7))

print('np.shape(SOHO_grand_valid_set_1):', np.shape(SOHO_grand_valid_set_1))
#print(len(SOHO_grand_valid_set_1) + len(SOHO_grand_valid_set_2) + len(SOHO_grand_valid_set_3)) #+ len(SOHO_grand_valid_set_4) + len(SOHO_grand_valid_set_5) + len(SOHO_grand_valid_set_6) + len(SOHO_grand_valid_set_7))

print('np.shape(SOHO_grand_test_set_1):', np.shape(SOHO_grand_test_set_1))
#print(len(SOHO_grand_test_set_1) + len(SOHO_grand_test_set_2) + len(SOHO_grand_test_set_3)) #+ len(SOHO_grand_test_set_4) + len(SOHO_grand_test_set_5) + len(SOHO_grand_test_set_6) + len(SOHO_grand_test_set_7))


Bz_train_set_len = len(Bz_train_set)
Bz_valid_set_len = len(Bz_valid_set)
Bz_test_set_len = len(Bz_test_set)

print('len(Bz_train_set):', Bz_train_set_len)
print('len(Bz_valid_set):', Bz_valid_set_len)
print('len(Bz_test_set):', Bz_test_set_len)
print('total data size:', Bz_train_set_len + Bz_valid_set_len + Bz_test_set_len)

Calculating threshold Bz values for MDI only family training set. For consistency will apply these Bz values to binarize the validation and test set within MDI only as well as the other two experiments (MDI + Lasco C2 + EIT195, and all 7 SOHO products together).

In [ ]:
'''
ONLY USE IF ONLY HAVE MDI PRODUCT.

PERFORM ONLY FOR MDI_ONLY TRAINING SET
'''

thresholds = list(np.arange(5,30,5)/100.)
print('thresholds:', thresholds)

Bz_train_set_sorted = np.sort(Bz_train_set) #most negative to least negative values

p_val_thres = np.round(np.arange(len(Bz_train_set_sorted)) / (len(Bz_train_set_sorted) - 1.),4)

ind_p = [np.where(p_val_thres == (elem + np.min(np.abs(p_val_thres - elem))))[0][0] for elem in thresholds]

Bz_train_set_sorted_thresholds = [Bz_train_set_sorted[elem] for elem in ind_p]
print('Bz_train_set_sorted_thresholds:', Bz_train_set_sorted_thresholds)

Bz values are -11.6, -8.91, -7.76, -6.94, and -6.27 nT @ (5,10,15,20,25)% threshold obtained from the MDI only experiment.

These thresholds are fixed for all three experiments. 

Binarize the ground truth labels accordingly

In [ ]:
'''
Binarize training, validation and test sets

Compute Class Weights
'''

Bz_train_per_threshold_list = []
Bz_valid_per_threshold_list = []
Bz_test_per_threshold_list = []

class_weights_list = []

Bz_thresholds_on_train_MDIonly = [-6.27, -6.94, -7.76, -8.91, -11.6]

for val in Bz_thresholds_on_train_MDIonly:
    
    Bz_train_set_copy = np.copy(Bz_train_set)
    Bz_valid_set_copy = np.copy(Bz_valid_set)
    Bz_test_set_copy = np.copy(Bz_test_set)
    
    ind1_train = np.where(np.array(Bz_train_set_copy) <= val)[0]
    #print('len(ind1_train):', len(ind1_train))
    ind0_train = np.where(np.array(Bz_train_set_copy) > val)[0]
    #print('len(ind0_train):', len(ind0_train))
    
    ind1_valid = np.where(np.array(Bz_valid_set_copy) <= val)[0]
    #print('len(ind1_valid):', len(ind1_valid))
    ind0_valid = np.where(np.array(Bz_valid_set_copy) > val)[0]
    #print('len(ind0_valid):', len(ind0_valid))
    
    ind1_test = np.where(np.array(Bz_test_set_copy) <= val)[0]
    #print('len(ind1_test):', len(ind1_test))
    ind0_test = np.where(np.array(Bz_test_set_copy) > val)[0]
    #print('len(ind0_test):', len(ind0_test))
    
    
    Bz_train_set_copy[ind1_train] = 1 #the events
    Bz_train_set_copy[ind0_train] = 0 #the non-events
    
    Bz_valid_set_copy[ind1_valid] = 1 #the events
    Bz_valid_set_copy[ind0_valid] = 0 #the non-events
    
    Bz_test_set_copy[ind1_test] = 1 #the events
    Bz_test_set_copy[ind0_test] = 0 #the non-events    
    
    Bz_train_per_threshold_list += list(Bz_train_set_copy)
    Bz_valid_per_threshold_list += list(Bz_valid_set_copy)
    Bz_test_per_threshold_list += list(Bz_test_set_copy)
    
    class_weights_train = class_weight.compute_class_weight('balanced', np.unique(Bz_train_set_copy), Bz_train_set_copy)
    class_weights_list.append(class_weights_train)

print('class_weights_list:', class_weights_list)

print('np.shape(Bz_train_per_threshold_list):', np.shape(Bz_train_per_threshold_list)) 
print('np.shape(Bz_valid_per_threshold_list):', np.shape(Bz_valid_per_threshold_list)) 
print('np.shape(Bz_test_per_threshold_list):', np.shape(Bz_test_per_threshold_list))

With the data ready and loaded, we are ready to conduct the three ML experiments with a deep CNN.

In [ ]:
print('TF version:', tf.__version__) ### check the TF API for your version of TF

# NEED THIS TO BE ABLE TO PEER INSIDE WITH .NUMPY() and to return results immediately
#tf.enable_eager_execution()
#tf.executing_eagerly()


### SET THE RANDOM SEED
tf.random.set_random_seed(1234) #this for TF version 1.12.0
np.random.seed(0)

### THIS IS A TEST TO MAKE SURE THAT "tf.random.set_random_seed(1234)" INPUT ABOVE SETS THE INITIALIZER 
initializer = tf.keras.initializers.glorot_uniform(seed=None)
values = initializer(shape=(3, 3))
print(tf.Session().run(values)) #.eval()
print('np.random.rand(3,2):', np.random.rand(3,2))

In [ ]:
'''
Currently the learning rate is a constant
'''

### epochs set to 1 as a test. Change this! 

epochs = 1 #100
lr = 0.01 #.01 for SGD optimizer, .001 for Adam optimizer
mini_batch = 64
img_size = np.shape(data)[2]
pool_size_factor = int(img_size/32) #to get the input image dimension down to size 32 x 32 for this example network
print('epochs:', epochs, 'learning rate:', lr, 'mini_batch size:', mini_batch, 'image y-dim:', img_size, 'pool_size_factor:', pool_size_factor)

In [ ]:
'''
Optional: freeing up memory held by these variables
'''

cube_list = None 
omni2_Bzmin_minus_mean_3to5days_ahead = None 
data = None

**Base unit block architecture for all 7 products: will have seven seperate models. Each architecture is fixed per product! Trainable = True, so layers not frozen! Seed=None in kernel_initializer because it has already been fixed by tf.random.set_random_seed(1234) above.**

In [ ]:
'''
Setting the three-layer architecture to be input into tf.keras.Sequential()
'''

architecture_list = []

for prodname in product_name_list: 
    
    architecture = [
        tf.keras.layers.InputLayer(
                input_shape = (1, img_size, img_size),
                name = f'conv2d0_{prodname}_input'), 

        tf.keras.layers.MaxPooling2D(pool_size = (pool_size_factor, pool_size_factor), strides = pool_size_factor, padding = 'same', data_format = 'channels_first', name = f'maxpool_{prodname}'), 
        tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), use_bias=False, data_format = 'channels_first', padding = 'same', trainable = True, kernel_initializer = tf.keras.initializers.glorot_uniform(seed=None), name = f'conv2d1_{prodname}'),
        tf.keras.layers.BatchNormalization(axis=1, trainable = True, name = f'BN1_{prodname}'),
        tf.keras.layers.ReLU(name = f'ReLU1_{prodname}'), 
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'same', data_format = 'channels_first', name = f'maxpl2_{prodname}'), 

        tf.keras.layers.Conv2D(32, (3, 3), use_bias=False, padding = 'same', data_format = 'channels_first', trainable = True, kernel_initializer = tf.keras.initializers.glorot_uniform(seed=None), name = f'conv2d2_{prodname}'),
        tf.keras.layers.BatchNormalization(axis=1, trainable = True, name = f'BN2_{prodname}'),
        tf.keras.layers.ReLU(name = f'ReLU2_{prodname}'), 
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'same', data_format = 'channels_first', name = f'maxpl3_{prodname}'),

        tf.keras.layers.Conv2D(16, (3, 3), use_bias=False, padding = 'same', data_format = 'channels_first', trainable = True, kernel_initializer = tf.keras.initializers.glorot_uniform(seed=None), name = f'conv2d3_{prodname}'),
        tf.keras.layers.BatchNormalization(axis=1, trainable = True, name = f'BN3_{prodname}'),
        tf.keras.layers.ReLU(name = f'ReLU3_{prodname}'),
        tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = 2, padding = 'same', data_format = 'channels_first', name = f'maxpl4_{prodname}'),

        tf.keras.layers.Flatten(name = f'flatten_{prodname}')
        ]
    
    architecture_list.append(architecture)

Create the Keras model

In [ ]:
'''
Create the Keras model

Need to adjust input according to if haave 1, 3, or 7 SOHO products by uncommenting the corresponding
'''

model_pre_list = [] #this is in the case where want to load weights later
model_pre_input_list = []
model_pre_output_list = []
for elem in range(len(product_name_list)):
    print('elem:', elem)
    model_pre = tf.keras.Sequential(architecture_list[elem]) 
    model_pre_list.append(model_pre)
    model_pre_input_list.append(model_pre.input)
    model_pre_output_list.append(model_pre.output)

#For 1 product:
#merged_pre_model = model_pre.output

#For 3 and 7 products:
merged_pre_model = tf.keras.layers.concatenate(model_pre_output_list, name ='concat')


pred1densepre_BN = tf.keras.layers.BatchNormalization(renorm=False, trainable = True, name = 'BN_concat')(merged_pre_model)
pred1dense = tf.keras.layers.Dense(units = 128, use_bias=False, trainable = True, kernel_regularizer = tf.keras.regularizers.l2(0.), kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0), name = 'dense1')(pred1densepre_BN)
pre1dense_ReLU1 = tf.keras.layers.ReLU(name = 'ReLU4')(pred1dense) #256 #128
pre1dense_ReLU1_dropout1 = tf.keras.layers.Dropout(rate=0.6, name = 'drop1')(pre1dense_ReLU1)
pred2dense = tf.keras.layers.Dense(units = 32, activation = 'linear', use_bias=False, trainable = True, kernel_regularizer = tf.keras.regularizers.l2(0.), kernel_initializer =   tf.keras.initializers.glorot_uniform(seed=None), name = 'dense2')(pre1dense_ReLU1_dropout1)
pred2dense_ReLU2 = tf.keras.layers.ReLU(name = 'ReLU5')(pred2dense) #64 #32
pred2dense_ReLU2_dropout2 = tf.keras.layers.Dropout(rate=0.6, name = 'drop2')(pred2dense_ReLU2)
pred3dense = tf.keras.layers.Dense(units = 1, activation = 'sigmoid', use_bias=True, trainable = True, kernel_regularizer = tf.keras.regularizers.l2(0.), kernel_initializer = tf.keras.initializers.glorot_uniform(seed=None), name = 'dense3')(pred2dense_ReLU2_dropout2)

# THE ACTUAL AUGMENETED MODEL
model = tf.keras.models.Model(model_pre_input_list, pred3dense)

# Clearing memory
model_pre_input_list = None
model_pre_output_list = None

# Create model summary
model.summary()

In [ ]:
# Compile the model with some loss function and optimizer #0.01 is default lr for SGD 

model.compile(
    loss = tf.keras.losses.binary_crossentropy, 
    optimizer = tf.keras.optimizers.SGD(lr = lr, decay = 1e-3, momentum=0.8, nesterov=False),
    metrics = ['accuracy'] 
)

In [ ]:
 '''
Enable early stopping and saving of model checkpoints
'''

experiment_dir = os.getcwd() #or user defined

tb = tf.keras.callbacks.TensorBoard(
    log_dir = experiment_dir,
    write_graph = True,
    write_images = True)

cp = tf.keras.callbacks.ModelCheckpoint(
    experiment_dir+"/model-weights-{epoch:02d}.hdf5", 
    monitor = 'val_loss', 
    verbose = 0, 
    save_best_only = True, 
    save_weights_only = False, 
    mode = 'auto', period = 10)

es = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None) 

In [ ]:
'''
Train, Validation, and Test labels per each of the five thresholds
'''

per_threshold_given_products_train_labels = np.array_split(Bz_train_per_threshold_list, len(Bz_thresholds_on_train_MDIonly)) 
per_threshold_given_products_valid_labels = np.array_split(Bz_valid_per_threshold_list, len(Bz_thresholds_on_train_MDIonly)) 
per_threshold_given_products_test_labels = np.array_split(Bz_test_per_threshold_list, len(Bz_thresholds_on_train_MDIonly)) 

print('train labels for all threholds:', np.shape(per_threshold_given_products_train_labels))
print('validation labels for all threholds:',np.shape(per_threshold_given_products_valid_labels))
print('test labels for all threholds:', np.shape(per_threshold_given_products_test_labels))

In [ ]:
'''
Running CNN network for the specified number of epochs
followed by Model fitting on the training and validation sets and Model predictions on the test set
Obtain TSS and MCC scores

NEED TO ADJUST FOR 1,3, OR 7 PRODUCTS
'''

for exper, thres in tqdm(enumerate(Bz_thresholds_on_train_MDIonly)): #outerloop to perform an ML experiment per threshold
    print(f'results for threshold {thres}')
    print('class_weights_list[exper]:', class_weights_list[exper])
    model.fit(
        
        #For 1 product:
        #x = SOHO_grand_train_set_1_normed, 
        
        #for all 3 products:
        #x = list((SOHO_grand_train_set_1_normed, SOHO_grand_train_set_2_normed, SOHO_grand_train_set_3_normed)),
        
        #for all 7 products:
        x = list((SOHO_grand_train_set_1_normed, SOHO_grand_train_set_2_normed, SOHO_grand_train_set_3_normed, SOHO_grand_train_set_4_normed, SOHO_grand_train_set_5_normed, SOHO_grand_train_set_6_normed, SOHO_grand_train_set_7_normed)),
        
        y = per_threshold_given_products_train_labels[exper], 
        epochs = epochs,
        batch_size = mini_batch,
        shuffle = True,
        initial_epoch=0,
        steps_per_epoch = None,
        verbose = 1,
        validation_data = (
        
        #For 1 product:
        #SOHO_grand_valid_set_1_normed,per_threshold_given_products_valid_labels[exper]),
        
        #For all 3 products:
        #list((SOHO_grand_valid_set_1_normed, SOHO_grand_valid_set_2_normed, SOHO_grand_valid_set_3_normed)), per_threshold_given_products_valid_labels[exper]),
        
        #For all 7 products:
        list((SOHO_grand_valid_set_1_normed, SOHO_grand_valid_set_2_normed, SOHO_grand_valid_set_3_normed, SOHO_grand_valid_set_4_normed, SOHO_grand_valid_set_5_normed, SOHO_grand_valid_set_6_normed, SOHO_grand_valid_set_7_normed)), per_threshold_given_products_valid_labels[exper]),
        
        class_weight = class_weights_list[exper],
        validation_steps = None,
        callbacks = [tb, cp, es] #so with early stopping!
    )
    #Having the model make the predictions
    predictions_test_data = model.predict(
        
        #For 1 product:
        #SOHO_grand_test_set_1_normed,
        
        #For all 3 products:
        #list((SOHO_grand_test_set_1_normed, SOHO_grand_test_set_2_normed, SOHO_grand_test_set_3_normed)),
        
        #For all 7 products:
        list((SOHO_grand_test_set_1_normed, SOHO_grand_test_set_2_normed, SOHO_grand_test_set_3_normed, SOHO_grand_test_set_4_normed, SOHO_grand_test_set_5_normed, SOHO_grand_test_set_6_normed, SOHO_grand_test_set_7_normed)),
        
        batch_size=mini_batch,
        steps = None,
        verbose = 1)

    predictions_train_data = model.predict(
        
        #For 1 product:
        #SOHO_grand_train_set_1_normed,
        
        #For all 3 products:
        #list((SOHO_grand_train_set_1_normed, SOHO_grand_train_set_2_normed, SOHO_grand_train_set_3_normed)),
        
        #For all 7 products:
        list((SOHO_grand_train_set_1_normed, SOHO_grand_train_set_2_normed, SOHO_grand_train_set_3_normed, SOHO_grand_train_set_4_normed, SOHO_grand_train_set_5_normed, SOHO_grand_train_set_6_normed, SOHO_grand_train_set_7_normed)),
        
        batch_size=mini_batch,
        steps = None,
        verbose = 1)

    predictions_valid_data = model.predict(
        
        #For 1 product:
        #SOHO_grand_valid_set_1_normed,
        
        #For all 3 products:
        #list((SOHO_grand_valid_set_1_normed, SOHO_grand_valid_set_2_normed, SOHO_grand_valid_set_3_normed)),        
        
        #For all 7 products:
        list((SOHO_grand_valid_set_1_normed, SOHO_grand_valid_set_2_normed, SOHO_grand_valid_set_3_normed, SOHO_grand_valid_set_4_normed, SOHO_grand_valid_set_5_normed, SOHO_grand_valid_set_6_normed, SOHO_grand_valid_set_7_normed)),
        
        batch_size=mini_batch,
        steps = None,
        verbose = 1)
    
    #### CHOOSING THE 50% CUTOFF PROBABILITY
    predictions_test_data_copy = np.copy(predictions_test_data)
    ind_0_pred_test = np.where(np.array(predictions_test_data_copy) < 0.5)[0]
    ind_1_pred_test = np.where(np.array(predictions_test_data_copy) >= 0.5)[0]
    predictions_test_data_copy[ind_0_pred_test] = 0
    predictions_test_data_copy[ind_1_pred_test] = 1
    #print('predictions_test_data_copy[0:10]:', predictions_test_data_copy[0:10])
    
    predictions_train_data_copy = np.copy(predictions_train_data)
    ind_0_pred_train = np.where(np.array(predictions_train_data_copy) < 0.5)[0]
    ind_1_pred_train = np.where(np.array(predictions_train_data_copy) >= 0.5)[0]
    predictions_train_data_copy[ind_0_pred_train] = 0
    predictions_train_data_copy[ind_1_pred_train] = 1
    #print('predictions_train_data_copy[0:10]:', predictions_train_data_copy[0:10])
    
    predictions_valid_data_copy = np.copy(predictions_valid_data)
    ind_0_pred_valid = np.where(np.array(predictions_valid_data_copy) < 0.5)[0]
    ind_1_pred_valid = np.where(np.array(predictions_valid_data_copy) >= 0.5)[0]
    predictions_valid_data_copy[ind_0_pred_valid] = 0
    predictions_valid_data_copy[ind_1_pred_valid] = 1
    #print('predictions_train_data_copy[0:10]:', predictions_valid_data_copy[0:10])
    
    confu_matrix_on_train = confusion_matrix(per_threshold_given_products_train_labels[exper], predictions_train_data_copy)
    print('confu_matrix_on_train:\n', confu_matrix_on_train)
    confu_matrix_on_valid = confusion_matrix(per_threshold_given_products_valid_labels[exper], predictions_valid_data_copy)
    print('confu_matrix_on_valid:\n', confu_matrix_on_valid)
    confu_matrix_on_test = confusion_matrix(per_threshold_given_products_test_labels[exper], predictions_test_data_copy)
    print('confu_matrix_on_test:\n', confu_matrix_on_test)
    
    matthews_on_test = matthews_corrcoef(per_threshold_given_products_test_labels[exper], predictions_test_data_copy)
    print('matthews_on_test:', matthews_on_test)
    tn, fp, fn, tp = confusion_matrix(per_threshold_given_products_test_labels[exper], predictions_test_data_copy).ravel()
    print('tn, fp, fn, tp:', tn, fp, fn, tp)
    TSS_on_test = tp/(tp + fn) - fp/(fp + tn)
    #matthews_on_test = ( (tp*tn)-(fp*fn) ) / np.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn)) #manual version
    #print('matthews_on_test:', matthews_on_test)
    print('TSS_on_test:', TSS_on_test)
    print('***next threshold***')

In [ ]:
# Save the model weights
model.save_weights(f"{experiment_dir}/model-{epochs}_Prods-{len(name_list)}_batches-{mini_batch}_size-{img_size}_{datetime.now().strftime('%Y-%m-%d-%H-%M')}.h5")

Preparing data for Gaussian Naive Bayes analysis
Approach is deterministic and so procedures applied to find feature combination on the train set directly hold
on the test set.

In [ ]:
'''
NEED TO ADJUST FOR 1,3, OR 7 PRODUCTS
'''

SOHO_grand_train_set_1_normed_squeeze = np.squeeze(SOHO_grand_train_set_1_normed)
SOHO_grand_test_set_1_normed_squeeze = np.squeeze(SOHO_grand_test_set_1_normed)

SOHO_grand_train_set_2_normed_squeeze = np.squeeze(SOHO_grand_train_set_2_normed)
SOHO_grand_test_set_2_normed_squeeze = np.squeeze(SOHO_grand_test_set_2_normed)

SOHO_grand_train_set_3_normed_squeeze = np.squeeze(SOHO_grand_train_set_3_normed)
SOHO_grand_test_set_3_normed_squeeze = np.squeeze(SOHO_grand_test_set_3_normed)

SOHO_grand_train_set_4_normed_squeeze = np.squeeze(SOHO_grand_train_set_4_normed)
SOHO_grand_test_set_4_normed_squeeze = np.squeeze(SOHO_grand_test_set_4_normed)

SOHO_grand_train_set_5_normed_squeeze = np.squeeze(SOHO_grand_train_set_5_normed)
SOHO_grand_test_set_5_normed_squeeze = np.squeeze(SOHO_grand_test_set_5_normed)

SOHO_grand_train_set_6_normed_squeeze = np.squeeze(SOHO_grand_train_set_6_normed)
SOHO_grand_test_set_6_normed_squeeze = np.squeeze(SOHO_grand_test_set_6_normed)

SOHO_grand_train_set_7_normed_squeeze = np.squeeze(SOHO_grand_train_set_7_normed)
SOHO_grand_test_set_7_normed_squeeze = np.squeeze(SOHO_grand_test_set_7_normed)

print('np.shape(SOHO_grand_train_set_1_normed_squeez):', np.shape(SOHO_grand_train_set_1_normed_squeeze))
print('np.shape(SOHO_grand_test_set_1_normed_squeeze):', np.shape(SOHO_grand_test_set_1_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_2_normed_squeez):', np.shape(SOHO_grand_train_set_2_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_2_normed_squeeze):', np.shape(SOHO_grand_test_set_2_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_3_normed_squeez):', np.shape(SOHO_grand_train_set_3_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_3_normed_squeeze):', np.shape(SOHO_grand_test_set_3_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_4_normed_squeez):', np.shape(SOHO_grand_train_set_4_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_4_normed_squeeze):', np.shape(SOHO_grand_test_set_4_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_5_normed_squeez):', np.shape(SOHO_grand_train_set_5_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_5_normed_squeeze):', np.shape(SOHO_grand_test_set_5_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_6_normed_squeez):', np.shape(SOHO_grand_train_set_6_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_6_normed_squeeze):', np.shape(SOHO_grand_test_set_6_normed_squeeze))

#print('np.shape(SOHO_grand_train_set_7_normed_squeez):', np.shape(SOHO_grand_train_set_7_normed_squeeze))
#print('np.shape(SOHO_grand_test_set_7_normed_squeeze):', np.shape(SOHO_grand_test_set_7_normed_squeeze))

In [ ]:
'''
NEED TO ADJUST FOR 1,3, OR 7 PRODUCTS
'''

data_array_1_tr = []
data_array_2_tr = []
data_array_3_tr = []
data_array_4_tr = []
data_array_5_tr = []
data_array_6_tr = []
data_array_7_tr = []

data_array_1_te = []
data_array_2_te = []
data_array_3_te = []
data_array_4_te = []
data_array_5_te = []
data_array_6_te = []
data_array_7_te = []


for l, elem in enumerate(SOHO_grand_train_set_1_normed_squeeze):
    data_array_1_tr.append(stats_mbfracdim(SOHO_grand_train_set_1_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_1_normed_squeeze[l]),-1))
    data_array_2_tr.append(stats_mbfracdim(SOHO_grand_train_set_2_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_2_normed_squeeze[l]),-1))
    data_array_3_tr.append(stats_mbfracdim(SOHO_grand_train_set_3_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_3_normed_squeeze[l]),-1))
    data_array_4_tr.append(stats_mbfracdim(SOHO_grand_train_set_4_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_4_normed_squeeze[l]),-1))
    data_array_5_tr.append(stats_mbfracdim(SOHO_grand_train_set_5_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_5_normed_squeeze[l]),-1))
    data_array_6_tr.append(stats_mbfracdim(SOHO_grand_train_set_6_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_6_normed_squeeze[l]),-1))
    data_array_7_tr.append(stats_mbfracdim(SOHO_grand_train_set_7_normed_squeeze[l],np.nanmean(SOHO_grand_train_set_7_normed_squeeze[l]),-1))

for l, elem in enumerate(SOHO_grand_test_set_1_normed_squeeze):
    data_array_1_te.append(stats_mbfracdim(SOHO_grand_test_set_1_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_1_normed_squeeze[l]),-1))
    data_array_2_te.append(stats_mbfracdim(SOHO_grand_test_set_2_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_2_normed_squeeze[l]),-1))
    data_array_3_te.append(stats_mbfracdim(SOHO_grand_test_set_3_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_3_normed_squeeze[l]),-1))
    data_array_4_te.append(stats_mbfracdim(SOHO_grand_test_set_4_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_4_normed_squeeze[l]),-1))
    data_array_5_te.append(stats_mbfracdim(SOHO_grand_test_set_5_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_5_normed_squeeze[l]),-1))
    data_array_6_te.append(stats_mbfracdim(SOHO_grand_test_set_6_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_6_normed_squeeze[l]),-1))
    data_array_7_te.append(stats_mbfracdim(SOHO_grand_test_set_7_normed_squeeze[l],np.nanmean(SOHO_grand_test_set_7_normed_squeeze[l]),-1))
    

print('np.shape(data_array_1_tr):', np.shape(data_array_1_tr))
#print('np.shape(data_array_2_tr):', np.shape(data_array_2_tr))
#print('np.shape(data_array_3_tr):', np.shape(data_array_3_tr))
#print('np.shape(data_array_4_tr):', np.shape(data_array_4_tr))
#print('np.shape(data_array_5_tr):', np.shape(data_array_5_tr))
#print('np.shape(data_array_6_tr):', np.shape(data_array_6_tr))
#print('np.shape(data_array_7_tr):', np.shape(data_array_7_tr))

print('np.shape(data_array_1_te):', np.shape(data_array_1_te))
#print('np.shape(data_array_2_te):', np.shape(data_array_2_te))
#print('np.shape(data_array_3_te):', np.shape(data_array_3_te))
#print('np.shape(data_array_4_te):', np.shape(data_array_4_te))
#print('np.shape(data_array_5_te):', np.shape(data_array_5_te))
#print('np.shape(data_array_6_te):', np.shape(data_array_6_te))
#print('np.shape(data_array_7_te):', np.shape(data_array_7_te))

In [ ]:
'''
*** SKIP IF 1 PRODUCT

NEED TO ADJUST FOR 3 or 7 PRODUCTS!

Generates all combinations of the max, min, mean, std, and frac features for the 3 (216) 
or 7 products (279,936) unique feature combinations

This will be used to reveal which combination was best in the Gaussian Naive Bayes analysis section

index_factor_7products = 400 can change to 10. 
This is to sample at a rate of one in every 400 for the seven product case
'''

#FOR 3 PRODUCTS
#data_tr = np.vstack((data_array_1_tr,data_array_2_tr,data_array_3_tr)) 
#data_te = np.vstack((data_array_1_te,data_array_2_te,data_array_3_te)) 

#FOR 7 PRODUCTS
data_tr = np.vstack((data_array_1_tr,data_array_2_tr,data_array_3_tr, data_array_4_tr, data_array_5_tr, data_array_6_tr, data_array_7_tr))
data_te = np.vstack((data_array_1_te,data_array_2_te,data_array_3_te, data_array_4_te, data_array_5_te, data_array_6_te, data_array_7_te))



# These 0-5 indices correspond, in this order, to the max, min, mean, std, frac, and 0 output by 
# the 'stats_mbfracdim()' function
# if hava added kurtosis and skewness then need to add these lines below following the pattern here

data_tr_max = data_tr.T[0]
data_tr_min = data_tr.T[1]
data_tr_mean = data_tr.T[2]
data_tr_std = data_tr.T[3]
data_tr_frac = data_tr.T[4]
data_tr_zero = data_tr.T[5]

ind_nan_frac_tr = np.where(data_tr_frac != data_tr_frac)[0]
data_tr_frac[ind_nan_frac_tr] = 0

data_te_max = data_te.T[0]
data_te_min = data_te.T[1]
data_te_mean = data_te.T[2]
data_te_std = data_te.T[3]
data_te_frac = data_te.T[4]
data_te_zero = data_te.T[5]

ind_nan_frac_te = np.where(data_te_frac != data_te_frac)[0]
data_te_frac[ind_nan_frac_te] = 0



product_stats_tr = []
product_stats_te = []

for l in range(len(name_list)):
    product_stats_tr.append((np.array_split(data_tr_max,len(name_list))[l], np.array_split(data_tr_min,len(name_list))[l], np.array_split(data_tr_mean,len(name_list))[l], np.array_split(data_tr_std,len(name_list))[l], np.array_split(data_tr_frac,len(name_list))[l], np.array_split(data_tr_zero,len(name_list))[l] ))
    product_stats_te.append((np.array_split(data_te_max,len(name_list))[l], np.array_split(data_te_min,len(name_list))[l], np.array_split(data_te_mean,len(name_list))[l], np.array_split(data_te_std,len(name_list))[l], np.array_split(data_te_frac,len(name_list))[l], np.array_split(data_te_zero,len(name_list))[l] ))


### ADJUST IF HAVE 3 OR 7 PRODUCTS
product_stats_tr_1 = np.array_split(product_stats_tr, len(name_list))[0][0]
product_stats_tr_2 = np.array_split(product_stats_tr, len(name_list))[1][0]
product_stats_tr_3 = np.array_split(product_stats_tr, len(name_list))[2][0]
product_stats_tr_4 = np.array_split(product_stats_tr, len(name_list))[3][0]
product_stats_tr_5 = np.array_split(product_stats_tr, len(name_list))[4][0]
product_stats_tr_6 = np.array_split(product_stats_tr, len(name_list))[5][0]
product_stats_tr_7 = np.array_split(product_stats_tr, len(name_list))[6][0]


product_stats_te_1 = np.array_split(product_stats_te, len(name_list))[0][0]
product_stats_te_2 = np.array_split(product_stats_te, len(name_list))[1][0]
product_stats_te_3 = np.array_split(product_stats_te, len(name_list))[2][0]
product_stats_te_4 = np.array_split(product_stats_te, len(name_list))[3][0]
product_stats_te_5 = np.array_split(product_stats_te, len(name_list))[4][0]
product_stats_te_6 = np.array_split(product_stats_te, len(name_list))[5][0]
product_stats_te_7 = np.array_split(product_stats_te, len(name_list))[6][0]


# For 3 products:
#product_stats_tr_all_combos = list(itertools.product(product_stats_tr_1,product_stats_tr_2,product_stats_tr_3)) 
#product_stats_te_all_combos = list(itertools.product(product_stats_te_1,product_stats_te_2,product_stats_te_3)) 


# Factor at which to subsample the 279936 combinations of one features from each of the 7 products 
index_factor_7products = 400 #100 #400 #10

# For 7 products:
product_stats_tr_all_combos = list(itertools.product(product_stats_tr_1,product_stats_tr_2,product_stats_tr_3, product_stats_tr_4, product_stats_tr_5, product_stats_tr_6, product_stats_tr_7))[::index_factor_7products]
product_stats_te_all_combos = list(itertools.product(product_stats_te_1,product_stats_te_2,product_stats_te_3, product_stats_te_4, product_stats_te_5, product_stats_te_6, product_stats_te_7))[::index_factor_7products]

print('np.shape(product_stats_tr_all_combos):', np.shape(product_stats_tr_all_combos))
print('np.shape(product_stats_te_all_combos):', np.shape(product_stats_te_all_combos))



#order of max, min, mean, std, frac dim, 0
AAA = ['max1','min1', 'mean1', 'std1','frac1', '0']
BBB = ['max2','min2', 'mean2', 'std2','frac2', '0']
CCC = ['max3','min3', 'mean3', 'std3','frac3', '0']
DDD = ['max4','min4', 'mean4', 'std4','frac4', '0']
EEE = ['max5','min5', 'mean5', 'std5','frac5', '0']
FFF = ['max6','min6', 'mean6', 'std6','frac6', '0']
GGG = ['max7','min7', 'mean7', 'std7','frac7', '0']


print('np.shape(list(itertools.product(AAA,BBB,CCC))):', np.shape(list(itertools.product(AAA,BBB,CCC))))
#print('np.shape(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products]):', np.shape(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products]))

#for xs in list(itertools.product(AAA,BBB,CCC)): #,DDD,EEE,FFF,GGG)): #[-100:]:
#    print(xs)

In [ ]:
'''
*** SKIP IF 3 or 7 PRODUCTS

USE ONLY IF HAVE 1 PRODUCT

Generates all combinations of the max, min, mean, std, and frac features for MDI only for 31 unique 
feature combinations. This will be used to reveal which combination was best in the Gaussian Naive Bayes analysis section
'''

lst = ['max','min','mean','std','frac']
combs = []

for i in range(1, len(lst)+1):
    els = [list(x) for x in itertools.combinations(lst, i)]
    combs.extend(els)

print(np.shape(combs))

for combi in combs:
    print(combi)

    

#since don't need the end zero in this case which is the 6th row
one_prod_tr = np.array(data_array_1_tr).T[0:5] 
one_prod_te = np.array(data_array_1_te).T[0:5] 

print('np.shape(one_prod_tr):', np.shape(one_prod_tr))
print('np.shape(one_prod_te):', np.shape(one_prod_te))

product_stats_tr_all_combos_pre = []
product_stats_te_all_combos_pre = []

for i in range(1, len(one_prod_tr)+1):
    combs_tr = [np.array(x) for x in itertools.combinations(one_prod_tr, i)]
    product_stats_tr_all_combos_pre.extend(combs_tr)

for i in range(1, len(one_prod_te)+1):
    combs_te = [np.array(x) for x in itertools.combinations(one_prod_te, i)]
    product_stats_te_all_combos_pre.extend(combs_te)


product_stats_tr_all_combos = []  
product_stats_te_all_combos = []

for combi in product_stats_tr_all_combos_pre:
    product_stats_tr_all_combos.append(np.array_split(np.ravel(combi, order='F'),Bz_train_set_len))


for combi in product_stats_te_all_combos_pre:
    product_stats_te_all_combos.append(np.array_split(np.ravel(combi, order='F'),Bz_test_set_len))

In [ ]:
'''
This is the order of the SOHO Products for each experiment
It is this order which determines the singleton features found
'''

print('product_name_list:', product_name_list)

In [ ]:
'''
Gaussian Naive Bayes Analysis
TSS and MCC scores per threshold

NEED TO ADJUST FOR 1,3, OR 7 PRODUCTS in six places!
'''

GNB_MCC_on_test_all5thres = []
GNB_TSS_on_test_all5thres = []
tfnp = []

for thres in range(len(Bz_thresholds_on_train_MDIonly)):
    GNB_MCC_on_test_all5thres_local = []
    GNB_TSS_on_test_all5thres_local = []
    tfnp_local = []
    
    class_weights_prep = np.ones(Bz_train_set_len)
    class_weights_prep[np.where(per_threshold_given_products_train_labels[thres] == 0)[0]] = class_weights_list[thres][0]
    class_weights_prep[np.where(per_threshold_given_products_train_labels[thres] == 1)[0]] = class_weights_list[thres][1]    
    
    for l,combo in enumerate(product_stats_tr_all_combos):
        
        # For 1 product:
        #combo_zip = combo
        
        # For 3 products:
        #combo_zip = list(zip(combo[0],combo[1],combo[2])) 
        
        # For 7 products:
        combo_zip = list(zip(combo[0],combo[1],combo[2],combo[3],combo[4],combo[5],combo[6]))
        
        clf_GaussianNB = GaussianNB()
        clf_GaussianNB.fit(combo_zip, per_threshold_given_products_train_labels[thres], class_weights_prep)

        # For 1 product:
        #combo_zip_test = product_stats_te_all_combos[l] #for MDI (31, 1227)
        
        # For 3 products:
        #combo_zip_test = list(zip(product_stats_te_all_combos[l][0],product_stats_te_all_combos[l][1],product_stats_te_all_combos[l][2])) 
        
        # For 7 products:
        combo_zip_test = list(zip(product_stats_te_all_combos[l][0],product_stats_te_all_combos[l][1],product_stats_te_all_combos[l][2],product_stats_te_all_combos[l][3],product_stats_te_all_combos[l][4],product_stats_te_all_combos[l][5],product_stats_te_all_combos[l][6]))
        
        GNB_pred_on_test = clf_GaussianNB.predict(combo_zip_test)
        GNB_matthews_on_test = matthews_corrcoef(per_threshold_given_products_test_labels[thres],GNB_pred_on_test)
        tn_g, fp_g, fn_g, tp_g = confusion_matrix(per_threshold_given_products_test_labels[thres],GNB_pred_on_test).ravel()
        tfnp.append([tn_g, fp_g, fn_g, tp_g])
        GNB_TSS_on_test = tp_g/(tp_g + fn_g) - fp_g/(fp_g + tn_g)
        #GNB_matthews_on_test = ( (tp_g*tn_g)-(fp_g*fn_g) ) / np.sqrt((tp_g + fp_g)*(tp_g + fn_g)*(tn_g + fp_g)*(tn_g + fn_g)) #manual version
        #print('GNB_matthews_on_test:', GNB_matthews_on_test)
        #if GNB_matthews_on_test != GNB_matthews_on_test:
        #    GNB_matthews_on_test = 0
        GNB_MCC_on_test_all5thres.append(GNB_matthews_on_test)
        GNB_TSS_on_test_all5thres.append(GNB_TSS_on_test)
        
        GNB_MCC_on_test_all5thres_local.append(GNB_matthews_on_test)
        GNB_TSS_on_test_all5thres_local.append(GNB_TSS_on_test)
        
        tfnp_local.append([tn_g, fp_g, fn_g, tp_g])
    
    ###
    print(f'threshold{thres+1}')
    print('np.unique(class_weights_prep):', np.unique(class_weights_prep))
    print('\n')
    
    ###
    GNB_MCC_on_test_all5thres_local_max = np.max(GNB_MCC_on_test_all5thres_local)
    print('GNB_MCC_on_test_all5thres_local_max:', GNB_MCC_on_test_all5thres_local_max)
    ind_MCC_on_test_max_local = np.where(GNB_MCC_on_test_all5thres_local == GNB_MCC_on_test_all5thres_local_max)[0][0]
    print('ind_MCC_on_test_max_local:', ind_MCC_on_test_max_local)
    print('tn_g, fp_g, fn_g, tp_g tfnp_local[ind_MCC_on_test_max_local]:', tfnp_local[ind_MCC_on_test_max_local])
    
    ### *** NEED TO UPDATE THIS LINE IF 1, 3, OR 7 PRODUCTS *** ###
    
    # For 1 product:
    #print('combs[ind_MCC_on_test_max_local]:', combs[ind_MCC_on_test_max_local])
    
    ### *** NEED TO CHANGE BETWEEN 3 AND 7 PRODUCTS *** ###
    
    # For 3 products:
    #print(list(itertools.product(AAA,BBB,CCC))[ind_MCC_on_test_max_local])
    
    # For 7 products:
    print(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products][ind_MCC_on_test_max_local]) 

    
    print('TSS for MCC global max local:', GNB_TSS_on_test_all5thres_local[ind_MCC_on_test_max_local])
    
    ###
    GNB_TSS_on_test_all5thres_local_max = np.max(GNB_TSS_on_test_all5thres_local)
    print('GNB_TSS_on_test_all5thres_local_max:', GNB_TSS_on_test_all5thres_local_max)
    ind_TSS_on_test_max_local = np.where(GNB_TSS_on_test_all5thres_local == GNB_TSS_on_test_all5thres_local_max)[0][0]
    print('ind_TSS_on_test_max_local:', ind_TSS_on_test_max_local)
    print('tn_g, fp_g, fn_g, tp_g tfnp_local[ind_TSS_on_test_max_local]:', tfnp_local[ind_TSS_on_test_max_local])    
    
    
    ### *** NEED TO UPDATE THIS LINE IF 1, 3, OR 7 PRODUCTS *** ###
    
    # For 1 product:
    #print('combs[ind_TSS_on_test_max_local]:', combs[ind_TSS_on_test_max_local])
    
    
    ### *** NEED TO CHANGE BETWEEN 3 AND 7 PRODUCTS *** ###
    
    # For 3 products:
    #print(list(itertools.product(AAA,BBB,CCC))[ind_TSS_on_test_max_local])
    
    # For 7 products:
    print(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products][ind_TSS_on_test_max_local])
    
    print('MCC for TSS global max local:', GNB_MCC_on_test_all5thres_local[ind_TSS_on_test_max_local])
    
    
    ###
    print(f'MCC max on threshold{thres+1}:', GNB_MCC_on_test_all5thres_local_max)
    print(f'TSS max on threshold{thres+1}:', GNB_TSS_on_test_all5thres_local_max)
    print('\n')

    
print('\n')
GNB_MCC_on_test_all5thres_max = np.max(GNB_MCC_on_test_all5thres)
print('global max MCC:', GNB_MCC_on_test_all5thres_max)
ind_MCC_on_test_max = np.where(GNB_MCC_on_test_all5thres == GNB_MCC_on_test_all5thres_max)[0][0] % len(product_stats_tr_all_combos)
print('ind_MCC_on_test_max:', ind_MCC_on_test_max)
print('tn_g, fp_g, fn_g, tp_g tfnp[ind_MCC_on_test_max]:', tfnp[ind_MCC_on_test_max])


### *** NEED TO UPDATE THIS LINE IF 1, 3, OR 7 PRODUCTS *** ###

# For 1 product:
#print('combs[ind_MCC_on_test_max]:', combs[ind_MCC_on_test_max])


### *** NEED TO CHANGE BETWEEN 3 AND 7 PRODUCTS *** ###

# For 3 products:
#print(list(itertools.product(AAA,BBB,CCC))[ind_MCC_on_test_max])

# For 7 products:
print(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products][ind_MCC_on_test_max])

print('TSS for MCC global max:', GNB_TSS_on_test_all5thres[ind_MCC_on_test_max])

print('\n')
GNB_TSS_on_test_all5thres_max = np.max(GNB_TSS_on_test_all5thres)
print('global max TSS:', GNB_TSS_on_test_all5thres_max)
ind_TSS_on_test_max = np.where(GNB_TSS_on_test_all5thres == GNB_TSS_on_test_all5thres_max)[0][0] % len(product_stats_tr_all_combos)
print('ind_TSS_on_test_max:', ind_TSS_on_test_max)
print('tn_g, fp_g, fn_g, tp_g tfnp[ind_TSS_on_test_max]:', tfnp[ind_TSS_on_test_max])

### *** NEED TO UPDATE THIS LINE IF 1, 3, OR 7 PRODUCTS *** ###

# For 1 product:
#print('combs[ind_TSS_on_test_max]:', combs[ind_TSS_on_test_max])

### *** NEED TO CHANGE BETWEEN 3 AND 7 PRODUCTS *** ###

# For 3 products
#print(list(itertools.product(AAA,BBB,CCC))[ind_TSS_on_test_max])

# For 7 products
print(list(itertools.product(AAA,BBB,CCC,DDD,EEE,FFF,GGG))[::index_factor_7products][ind_TSS_on_test_max])

print('MCC for TSS global max:', GNB_MCC_on_test_all5thres[ind_TSS_on_test_max])